In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms



In [7]:
#This chunk prepares the image data for input into a neural network.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [8]:
train_data = torchvision.datasets.CIFAR10(root = './data', train = True, transform=transform, download = True)
test_data = torchvision.datasets.CIFAR10(root = './data', train = False, transform=transform, download = True) 

#provide the data in batches
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 32, shuffle = True, num_workers = 2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 32, shuffle = True, num_workers = 2)


Files already downloaded and verified
Files already downloaded and verified


In [9]:
image, label = train_data[0]

In [10]:
image.size()

torch.Size([3, 32, 32])

In [11]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [13]:

class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)  #our new shape is (12,28, 28)
        self.pool = nn.MaxPool2d(2,2) #(12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) #(24, 10, 10) -> (24, 5, 5) -> Flatten(24*5*5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x




In [16]:
#We are now going to train the model

#this is our training loop
#instantiate neural network model by creating an object of the NeuralNet class.
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)
for epoch in range(30):
    print(f'Training epoch {epoch}...')
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss =+ loss.item()
    print(f'loss: {running_loss / len(train_loader):.4f}')


Training epoch 0...
loss: 0.0013
Training epoch 1...
loss: 0.0009
Training epoch 2...
loss: 0.0009
Training epoch 3...
loss: 0.0008
Training epoch 4...
loss: 0.0008
Training epoch 5...
loss: 0.0007
Training epoch 6...
loss: 0.0007
Training epoch 7...
loss: 0.0009
Training epoch 8...
loss: 0.0005
Training epoch 9...
loss: 0.0008
Training epoch 10...
loss: 0.0006
Training epoch 11...
loss: 0.0005
Training epoch 12...
loss: 0.0003
Training epoch 13...
loss: 0.0005
Training epoch 14...
loss: 0.0004
Training epoch 15...
loss: 0.0003
Training epoch 16...
loss: 0.0007
Training epoch 17...
loss: 0.0005
Training epoch 18...
loss: 0.0007
Training epoch 19...
loss: 0.0004
Training epoch 20...
loss: 0.0007
Training epoch 21...
loss: 0.0005
Training epoch 22...
loss: 0.0003
Training epoch 23...
loss: 0.0001
Training epoch 24...
loss: 0.0002
Training epoch 25...
loss: 0.0003
Training epoch 26...
loss: 0.0003
Training epoch 27...
loss: 0.0003
Training epoch 28...
loss: 0.0004
Training epoch 29...
los

In [18]:
torch.save(net.state_dict(), 'trained_net.pth')

In [19]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [20]:
#Evaluate on test data
correct = 0
total = 0
#sets the model to evaluation mode
net.eval()
#disables gradient calculation
with torch.no_grad():
    #The loop iterates over the batches in the test_loader, which contains the test dataset.
    for data in test_loader:
        images, labels = data
        #The model processes the input images and produces a set of outputs (logits). Each output corresponds to the network's confidence in each class.
        outputs = net(images)
        #This function returns the maximum value and its index along the specified dimension
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 *correct/total

print(f'Accuracy: {accuracy}%')

Accuracy: 68.84%


In [24]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    #Adding a batch dimension allows the same code to handle both single-image and multi-image inputs
    image = image.unsqueeze(0)
    return image


image_paths = ['Airplane.jpg', 'Dog.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        # Passes the image through the neural network
        output = net(image)
        #Finds the index of the maximum value in the output tensor
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}') 

Prediction: plane
Prediction: dog
